# Regressione Lineare con Spark MLlib

In questo notebook vedremo come eseguire una semplice regressione lineare con il modulo MLlib di Spark. Il modello che andremo a creare avrà lo scopo di stimare il valore di un'abitazione utilizzando un set di proprietà:

* **CRIM** Tasso di criminalità per capita
* **ZN** Percentuale di terreni residenziali suddivisi in zone per lotti superiori a 25.000 sq.ft.
* **INDUS** Percentuale di ettari di attività non al dettaglio per città.
* **CHAS** Variabile dummy che indica la prossimità al fiume Charles.
* **NOX** Concentrazione di ossido d'azoto (parti per 10 milioni).
* **RM** Numero medio di stanze per abitazione
* **AGE** Percentuale di abitazione occupate costruite dopo il 1940
* **DIS** Media pesata delle distanze da 5 centri lavorativi di Boston.
* **RAD** Indice di accessibilità ad autostrade
* **TAX** Aliquota dell'imposta sulla proprietà a valore pieno in 10.000 USD.
* **PRATIO**** Rapporto studente-insegnante per città.
* **BLACK** 1000(Bk - 0.63)^2 dove Bk è la percentuale di abitanti di colore per città
* **LSTAT** Percentuale della popolazione povera
* **MEDV** Mediana del valore di abitazioni occupate in 1.000 USD.

## Importazione delle librerie

In [1]:
import os
import numpy as np
import pandas as pd

import pyspark.sql.functions
from pyspark.sql.types import *

## Inizializzazione di Spark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basic').getOrCreate()
from pyspark.sql.types import *

## Importazione del dataset

Il dataset che utilizzeremo è il **Boston Housing Dataset**

In [3]:
housing_df = spark.read.csv('HousingData.csv', header=True, inferSchema=True)

In [4]:
housing_df.show()

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
|   CRIM|  ZN|INDUS|CHAS|  NOX|   RM| AGE|   DIS|RAD|TAX|PTRATIO|     B|LSTAT|MEDV|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
|0.00632|  18| 2.31|   0|0.538|6.575|65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731|   0| 7.07|   0|0.469|6.421|78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729|   0| 7.07|   0|0.469|7.185|61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|0.03237|   0| 2.18|   0|0.458|6.998|45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|0.06905|   0| 2.18|   0|0.458|7.147|54.2|6.0622|  3|222|   18.7| 396.9|   NA|36.2|
|0.02985|   0| 2.18|   0|0.458| 6.43|58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|
|0.08829|12.5| 7.87|  NA|0.524|6.012|66.6|5.5605|  5|311|   15.2| 395.6|12.43|22.9|
|0.14455|12.5| 7.87|   0|0.524|6.172|96.1|5.9505|  5|311|   15.2| 396.9|19.15|27.1|
|0.21124|12.5| 7.87|   0|0.524|5.631| 100|6.0821|  5|311|   15.2|386.63|29.9

In [5]:
housing_df.printSchema()

root
 |-- CRIM: string (nullable = true)
 |-- ZN: string (nullable = true)
 |-- INDUS: string (nullable = true)
 |-- CHAS: string (nullable = true)
 |-- NOX: double (nullable = true)
 |-- RM: double (nullable = true)
 |-- AGE: string (nullable = true)
 |-- DIS: double (nullable = true)
 |-- RAD: integer (nullable = true)
 |-- TAX: integer (nullable = true)
 |-- PTRATIO: double (nullable = true)
 |-- B: double (nullable = true)
 |-- LSTAT: string (nullable = true)
 |-- MEDV: double (nullable = true)



E' necessario modificare alcuni tipi.

In [6]:
data_schema = [StructField('CRIM',FloatType(), True),
               StructField('ZN',FloatType(), True),
               StructField('INDUS',FloatType(), True),
               StructField('CHAS',IntegerType(), True),
               StructField('NOX',FloatType(), True),
               StructField('RM',FloatType(), True),
               StructField('AGE',FloatType(), True),
               StructField('DIS',FloatType(), True),
               StructField('RAD',IntegerType(), True),
               StructField('TAX',IntegerType(), True),
               StructField('PTRATIO',FloatType(), True),
               StructField('B',FloatType(), True),
               StructField('LSTAT',FloatType(), True),
               StructField('MEDV',FloatType(), True)]

schema = StructType(fields = data_schema)

In [7]:
housing_df = spark.read.csv('HousingData.csv', header=True, schema = schema)

In [8]:
housing_df.show()

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|   CRIM|  ZN|INDUS|CHAS|  NOX|   RM|  AGE|   DIS|RAD|TAX|PTRATIO|     B|LSTAT|MEDV|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| null|36.2|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|
|0.08829|12.5| 7.87|null|0.524|6.012| 66.6|5.5605|  5|311|   15.2| 395.6|12.43|22.9|
|0.14455|12.5| 7.87|   0|0.524|6.172| 96.1|5.9505|  5|311|   15.2| 396.9|19.15|27.1|
|0.21124|12.5| 7.87|   0|0.524|5.631|100.0|6.0821|  5|311|   15.2

## Preprocessing dei dati

In [9]:
housing_df.count()

506

### Valori nulli

Innanziutto verifichiamo la presenza di valori nulli

In [10]:
housing_df.filter(housing_df["CRIM"].isNull() | \
                  housing_df["ZN"].isNull() | \
                  housing_df["INDUS"].isNull() | \
                  housing_df["CHAS"].isNull() | \
                  housing_df["NOX"].isNull() | \
                  housing_df["RM"].isNull() | \
                  housing_df["AGE"].isNull() | \
                  housing_df["DIS"].isNull() | \
                  housing_df["RAD"].isNull() | \
                  housing_df["TAX"].isNull() | \
                  housing_df["PTRATIO"].isNull() | \
                  housing_df["B"].isNull() | \
                  housing_df["LSTAT"].isNull() | \
                  housing_df["MEDV"].isNull() \
                 ).count()

112

112 righe su 506 presentano almeno un valore nullo. Per ragioni di semplicità andiamo a rimuovere tali righe (in teoria sarebbe meglio fare risolvere la cosa diversamente)

In [11]:
housing_df = housing_df.filter(housing_df["CRIM"].isNotNull() & \
                              housing_df["ZN"].isNotNull() & \
                              housing_df["INDUS"].isNotNull() & \
                              housing_df["CHAS"].isNotNull() & \
                              housing_df["NOX"].isNotNull() & \
                              housing_df["RM"].isNotNull() & \
                              housing_df["AGE"].isNotNull() & \
                              housing_df["DIS"].isNotNull() & \
                              housing_df["RAD"].isNotNull() & \
                              housing_df["TAX"].isNotNull() & \
                              housing_df["PTRATIO"].isNotNull() & \
                              housing_df["B"].isNotNull() & \
                              housing_df["LSTAT"].isNotNull() & \
                              housing_df["MEDV"].isNotNull() \
                             )

In [12]:
housing_df.show()

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|   CRIM|  ZN|INDUS|CHAS|  NOX|   RM|  AGE|   DIS|RAD|TAX|PTRATIO|     B|LSTAT|MEDV|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|
|0.14455|12.5| 7.87|   0|0.524|6.172| 96.1|5.9505|  5|311|   15.2| 396.9|19.15|27.1|
|0.21124|12.5| 7.87|   0|0.524|5.631|100.0|6.0821|  5|311|   15.2|386.63|29.93|16.5|
|0.22489|12.5| 7.87|   0|0.524|6.377| 94.3|6.3467|  5|311|   15.2|392.52|20.45|15.0|
|0.11747|12.5| 7.87|   0|0.524|6.009| 82.9|6.2267|  5|311|   15.2

### VectorAssembler

Genero una lista con i nomi delle colonne che saranno le features del nostro modello, cioè tutte le colonne meno il target (MEDV).

In [13]:
columns = housing_df.columns[:-1]

La classe MLlib richiede che le features si trovino tutte all'interno di un unico vettore su di una colonna, possiamo creare questa rappresentazione utilizzando la classe VectorAssemlber di MLlib.

In [14]:
from pyspark.ml.feature import VectorAssembler

In [15]:
assembler = VectorAssembler(inputCols=columns, outputCol="features")

In [16]:
data_df = assembler.transform(housing_df)

In [17]:
data_df.show(n=3,truncate=False)

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|CRIM   |ZN  |INDUS|CHAS|NOX  |RM   |AGE |DIS   |RAD|TAX|PTRATIO|B     |LSTAT|MEDV|features                                                                                                                                                                                   |
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0.00632|18.0|2.31 |0   |0.538|6.575|65.2|4.09  |1  |296|15.3   |396.9 |4.98 |24.0|[0.006320000160485506,18.0,2.309999942779541,0.0,0.5379999876022339,6.574999809265137,65.199996948242

### Normalizzazione

E' buona norma portare le features in un range di valori comuni, questo processo può velocizzare anche di molto la fase di addestramento. Facciamolo utilizzando la **normalizzazione** che si esegue sottraendo il valore minimo e poi dividendo per la differenza tra valore massimo e valore minimo. Possiamo eseguire la normalizzazione con MLlib usando la classe **MinMaxScaler**.

In [18]:
from pyspark.ml.feature import MinMaxScaler

In [19]:
scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")
scaler_model = scaler.fit(data_df)
data_df = scaler_model.transform(data_df)

In [20]:
data_df.show(n=3)

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+--------------------+
|   CRIM|  ZN|INDUS|CHAS|  NOX|   RM| AGE|   DIS|RAD|TAX|PTRATIO|     B|LSTAT|MEDV|            features|     scaled_features|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+--------------------+
|0.00632|18.0| 2.31|   0|0.538|6.575|65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|[0.00632000016048...|[0.0,0.18,0.06781...|
|0.02731| 0.0| 7.07|   0|0.469|6.421|78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|[0.02731000073254...|[2.35922555448959...|
|0.02729| 0.0| 7.07|   0|0.469|7.185|61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|[0.02728999964892...|[2.35697748082166...|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+--------------------+
only showing top 3 rows



### Train set e Test set

Prossimo passo, dividere il DataFrame con le features preprocessate in due DataFrame, uno per l'**addestramento** e uno per il **testing** del modello, possiamo farlo utilizzando il metodo **randomSplit** all'interno della quale dobbiamo passare una lista con la percentuale di osservazioni da assegnare ad ognuno dei DataFrame.
Nel nostro caso assegnamo il 70% degli esempi al set di addestramento e il 30% al set di test.

In [21]:
train_df, test_df = data_df.randomSplit([0.7, 0.3])

In [22]:
print("%d esempi nel train set" % train_df.count())
print("%d esempi nel test set" % test_df.count())

281 esempi nel train set
113 esempi nel test set


In [23]:
test_df.show(3)

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+--------------------+
|   CRIM|  ZN|INDUS|CHAS|  NOX|   RM| AGE|   DIS|RAD|TAX|PTRATIO|     B|LSTAT|MEDV|            features|     scaled_features|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+--------------------+
|0.00632|18.0| 2.31|   0|0.538|6.575|65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|[0.00632000016048...|[0.0,0.18,0.06781...|
|0.01301|35.0| 1.52|   0|0.442|7.241|49.3|7.0379|  1|284|   15.5|394.74| 5.49|32.7|[0.01300999987870...|[7.51939869680761...|
|0.01311|90.0| 1.22|   0|0.403|7.249|21.9|8.6966|  5|226|   17.9|395.93| 4.81|35.4|[0.01310999970883...|[7.63179609949933...|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+--------------------+
only showing top 3 rows



## Generazione del modello

Ottimo ! Possiamo creare il modello di Regressione Lineare, usiamo la classe **LinearRegression**, all'interno del costruttore dovremo passare due parametri:

* **featuresCol**: il nome della colonna con le features
* **labelCol**: il nome della colonna con il target

In [24]:
from pyspark.ml.regression import LinearRegression

In [25]:
lr = LinearRegression(featuresCol="scaled_features", labelCol="MEDV")

Avviamo l'addestramento con il metodo fit, passando al suo interno il set di addetramento

In [26]:
model = lr.fit(train_df)

Abbiamo creato il nostro modello ! Ora verifichiamone la qualità testandolo su dati che non ha visto durante l'addestramento, possiamo farlo usando il test set e il metodo evaluate.

In [27]:
evaluation = model.evaluate(test_df)


Il metodo evaluate calcolerà diverse metriche che ci possono aiutare a comprendere la qualità del modello, vediamone alcune.

## Valutazione del modello

#### MAE - Mean Absolute Error (Errore medio assoluto)

L'errore medio assoluto consiste nella media della somma del valore assoluto degli errori.

$$ MAE = \frac{\sum_{i=1}^n |y_i-\hat{y}_i|}{n} $$

#### MSE - Mean Squared Error (Errore quadratico assoluto)

L'errore quadratico medio consiste nella media della somma degli errori al quadrato.

$$ MSE =  \frac{\sum_{i=1}^n (y_i-\hat{y}_i)^2}{n}$$

#### RMSE - Root Mean Squared Error (Radice dell'errore quadratico medio)

Il RMSE è la radice dell'errore quadratico medio, questa metrica indica mediamente di quanto il nostro modello si è sbagliato.

$$ RMSE =  \sqrt \frac{\sum_{i=1}^n (y_i-\hat{y}_i)^2}{n}$$

#### R2 - Coefficient of determination (Coefficiente di Determinazione)

In pratica R2 (pronuciato R Squared) è una versione standardizzata del MSE che torna un punteggio compreso tra 0 e 1 per il train set, mentre per il test set può assumere anche valori negativi. Essendo una funzione ma di scoring, un suo valore maggiore indica una qualità migliore del modello, il suo valore può essere così interpretato:

* R2_score < 0.3 il modello è inutile.
* 0.3 < R2_score < 0.5 il modello è scarso.
* 0.5 < R2_score < 0.7 il modello è discreto.
* 0.7 < R2_score < 0.9 il modello è buono.
* 0.9 < R2_score < 1 il modello è ottimo.
* R2_score = 1 molto probabilmente c'è un errore nel modello.

$$ R^2 = 1-\frac{RSS}{SST} $$

dove RSS è la somma dei quadrati residui:
$$ RSS = \sum_{i=1}^{N}(Y_i-\hat{Y}_i)^2 $$

ed SST è la somma dei quadrati totali:
$$ SST = \sum_{i=1}^{N}(Y_i-\bar{Y})^2 $$

In [28]:
print('Mean Absolute Error (Errore medio assoluto): ',evaluation.meanAbsoluteError)
print('Mean Squared Error (Errore quadratico assoluto): ', evaluation.meanSquaredError)
print('Root Mean Squared Error (Radice dell errore quadratico medio): ', evaluation.rootMeanSquaredError)
print('Coefficient of determination (Coefficiente di Determinazione): ', evaluation.r2)

Mean Absolute Error (Errore medio assoluto):  3.021670203672007
Mean Squared Error (Errore quadratico assoluto):  17.18885239612977
Root Mean Squared Error (Radice dell errore quadratico medio):  4.145944089846096
Coefficient of determination (Coefficiente di Determinazione):  0.7783887838114685


## Utilizzo del modello su nuovi dati

Ora che abbiamo un modello addestrato e funzionante testiamolo su nuovi dati, mettiamo caso che un'agenzia immobiliare ci abbia mandato un file CSV con le proprietà di 10 abitazioni per la quale stimare il prezzo usando il modello addestrato.

In [29]:
houses = spark.read.csv('houses.csv', header = True, inferSchema=True)

In [30]:
houses.show(5)

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+
|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio| black|lstat|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+
|0.05789|12.5| 6.07|   0|0.409|5.878|21.4| 6.498|  4|345|   18.9|396.21|  8.1|
|0.13554|12.5| 6.07|   0|0.409|5.594|36.8| 6.498|  4|345|   18.9| 396.9|13.09|
|0.08826| 0.0|10.81|   0|0.413|6.417| 6.6|5.2873|  4|305|   19.2|383.73| 6.72|
|0.09164| 0.0|10.81|   0|0.413|6.065| 7.8|5.2873|  4|305|   19.2|390.91| 5.52|
|0.19539| 0.0|10.81|   0|0.413|6.245| 6.2|5.2873|  4|305|   19.2|377.17| 7.54|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+
only showing top 5 rows



### Preprocessing

In [31]:
houses.count()

10

#### Check valori nulli

In [32]:
houses.select(houses['crim'].isNull() | \
              houses['zn'].isNull() | \
              houses['indus'].isNull() | \
              houses['chas'].isNull() | \
              houses['nox'].isNull() | \
              houses['rm'].isNull() | \
              houses['age'].isNull() | \
              houses['dis'].isNull() | \
              houses['rad'].isNull() | \
              houses['tax'].isNull() | \
              houses['ptratio'].isNull() | \
              houses['black'].isNull() | \
              houses['lstat'].isNull()).count()

10

Non ci sono valori nulli.

#### VectorAssembler

In [33]:
v_ass = VectorAssembler(inputCols = houses.columns, outputCol = 'features')

In [34]:
houses = v_ass.transform(houses)

In [35]:
houses.show()

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+--------------------+
|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio| black|lstat|            features|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+--------------------+
|0.05789|12.5| 6.07|   0|0.409|5.878|21.4| 6.498|  4|345|   18.9|396.21|  8.1|[0.05789,12.5,6.0...|
|0.13554|12.5| 6.07|   0|0.409|5.594|36.8| 6.498|  4|345|   18.9| 396.9|13.09|[0.13554,12.5,6.0...|
|0.08826| 0.0|10.81|   0|0.413|6.417| 6.6|5.2873|  4|305|   19.2|383.73| 6.72|[0.08826,0.0,10.8...|
|0.09164| 0.0|10.81|   0|0.413|6.065| 7.8|5.2873|  4|305|   19.2|390.91| 5.52|[0.09164,0.0,10.8...|
|0.19539| 0.0|10.81|   0|0.413|6.245| 6.2|5.2873|  4|305|   19.2|377.17| 7.54|[0.19539,0.0,10.8...|
|0.07896| 0.0|12.83|   0|0.437|6.273| 6.0|4.2515|  5|398|   18.7|394.92| 6.78|[0.07896,0.0,12.8...|
|0.09512| 0.0|12.83|   0|0.437|6.286|45.0|4.5026|  5|398|   18.7|383.23| 8.94|[0.09512,0.0,12.8...|


#### Standardizzazione

Applichiamo la normalizzazione, assicurandoci di applicare la stessa trasformazione che abbiamo applicato agli esempi di addestramento.

In [36]:
houses = scaler_model.transform(houses)

In [37]:
houses.show()

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+--------------------+--------------------+
|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio| black|lstat|            features|     scaled_features|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+--------------------+--------------------+
|0.05789|12.5| 6.07|   0|0.409|5.878|21.4| 6.498|  4|345|   18.9|396.21|  8.1|[0.05789,12.5,6.0...|[5.79634388521087...|
|0.13554|12.5| 6.07|   0|0.409|5.594|36.8| 6.498|  4|345|   18.9| 396.9|13.09|[0.13554,12.5,6.0...|[0.00145240170302...|
|0.08826| 0.0|10.81|   0|0.413|6.417| 6.6|5.2873|  4|305|   19.2|383.73| 6.72|[0.08826,0.0,10.8...|[9.20985880360672...|
|0.09164| 0.0|10.81|   0|0.413|6.065| 7.8|5.2873|  4|305|   19.2|390.91| 5.52|[0.09164,0.0,10.8...|[9.58976267005973...|
|0.19539| 0.0|10.81|   0|0.413|6.245| 6.2|5.2873|  4|305|   19.2|377.17| 7.54|[0.19539,0.0,10.8...|[0.00212510130086...|
|0.07896| 0.0|12.83|   0|0.437|6

### Predizione

A questo punto possiamo utilizzare il nostro modello per eseguire la predizione del valore degli immobili.

**N.B**: ovviamente il nostro modello considererà solo la colonna 'scaled_features' per eseguire la predizione

In [39]:
model.transform(houses).show()

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+--------------------+--------------------+------------------+
|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio| black|lstat|            features|     scaled_features|        prediction|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+--------------------+--------------------+------------------+
|0.05789|12.5| 6.07|   0|0.409|5.878|21.4| 6.498|  4|345|   18.9|396.21|  8.1|[0.05789,12.5,6.0...|[5.79634388521087...| 21.54255438338617|
|0.13554|12.5| 6.07|   0|0.409|5.594|36.8| 6.498|  4|345|   18.9| 396.9|13.09|[0.13554,12.5,6.0...|[0.00145240170302...| 18.16259090962968|
|0.08826| 0.0|10.81|   0|0.413|6.417| 6.6|5.2873|  4|305|   19.2|383.73| 6.72|[0.08826,0.0,10.8...|[9.20985880360672...|26.100158528591376|
|0.09164| 0.0|10.81|   0|0.413|6.065| 7.8|5.2873|  4|305|   19.2|390.91| 5.52|[0.09164,0.0,10.8...|[9.58976267005973...| 24.96409957353691|
|0.19539| 0.0|10.81|

Infatti facendo come di seguito si ottiene lo stesso risultato.

In [40]:
model.transform(houses.select('scaled_features')).show()

+--------------------+------------------+
|     scaled_features|        prediction|
+--------------------+------------------+
|[5.79634388521087...| 21.54255438338617|
|[0.00145240170302...| 18.16259090962968|
|[9.20985880360672...|26.100158528591376|
|[9.58976267005973...| 24.96409957353691|
|[0.00212510130086...|24.898058466120844|
|[8.16456118289281...|26.058668657213197|
|[9.98090629587525...|24.570810233414722|
|[0.00107013748710...| 23.75777310470015|
|[9.07610566719278...| 23.99936531344926|
|[4.61841710401928...|   21.905385648477|
+--------------------+------------------+

